# Workbench Setup

## Google Colab Installation

In [1]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Code specific to Google Colab
    print("Running in Google Colab")

    # Additional setup commands for Colab
    !pip install neuralforecast
    !pip install gymnasium
    !pip install QuantStats
else:
    # Code for other environments (e.g., VS Code)
    print("Running in another environment (e.g., VS Code)")

Running in another environment (e.g., VS Code)


In [2]:
if IN_COLAB:
    # Retrive required files
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/environments/stockenv.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/utilities/cleandata.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/utilities/data.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/utilities/epsilon_decay.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/utilities/agentperform.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/agent/ddqn.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/agent/random.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/reward/stockmarket.py            
    # Move all directories and files from content/raw.githubusercontent.com to content/
    !mv /content/raw.githubusercontent.com/* /content/

    # Delete the raw.githubusercontent.com directory
    !rm -rf /content/raw.githubusercontent.com

## Software Enviornment Setup

In [3]:
import os
import sys
import torch
import optuna
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import utilities.agentperform as agentperform
import utilities.cleandata as cln 
from utilities.epsilon_decay import linear_decay
from utilities.data import UniStockEnvDataStruct, TimesNetProcessing
from agents.ddqn import DdqnAgent
from rewards.stockmarket import future_profit, risk_reward
from environments.stockenv import ContinuousOHLCVEnv
from datetime import datetime
from neuralforecast.core import NeuralForecast
from neuralforecast.models import TimesNet
from neuralforecast.losses.numpy import mae, mse
import logging
# 
logging.getLogger("pytorch_lightning.utilities.rank_zero").addHandler(logging.NullHandler())
logging.getLogger("pytorch_lightning.accelerators.cuda").addHandler(logging.NullHandler())
os.environ['NIXTLA_ID_AS_COL'] = '1' # Prevent Warning 

def set_seed(seed):
    """Set seed for reproducibility."""
    # Python random module
    random.seed(seed)

    # NumPy
    np.random.seed(seed)

    # PyTorch
    torch.manual_seed(seed)

    # If you are using CUDA
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
        # Additional settings to force determinism in your operations:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False



# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the current device
    device = torch.cuda.current_device()
    print(f"GPU Name: {torch.cuda.get_device_name(device)}")
else:
    device = 'cpu'
    print("CUDA (GPU support) is not available. PyTorch is running on CPU.")


def decimal_to_text(decimal_number):
    # Remove the decimal point and convert to integer
    integer_part = int(decimal_number * 1000)
    # Convert the integer to text
    text_representation = str(integer_part)
    return text_representation

CUDA (GPU support) is not available. PyTorch is running on CPU.


## RL Environment Setup

In [4]:
RANDOM_SEED = 42
set_seed(RANDOM_SEED)

if not IN_COLAB:
    pwd = "C:/programming/MADDQN"
    sys.path.append(pwd)
    
    # Output Path Location for CSV export
    export_path = pwd + "/output_data"

# Input Data Location, File Name, Stock name for labels
input_url = 'https://raw.githubusercontent.com/CodeBeckZero/MADDQN/main/input_data'

stock_inputs ={'DJI':'^DJI_daily.csv',
               'NDAQ': '^IXIC_daily.csv',
               'SP500': '^SPX_daily.csv',
               'AAPL': 'AAPL_daily.csv',
               'AMZN': 'AMZN_daily.csv',
               'GOOGL': 'GOOGL_daily.csv',
               'MSFT': 'MSFT_daily.csv',
               'SINE': 'sine_wave_daily.csv',
               'FORD': 'F_daily.csv',
               'JNJ': 'JNJ_daily.csv',
               'NEE': 'NEE_daily.csv',
               'PFE': 'PFE_daily.csv',
               'TSLA': 'TSLA_daily.csv',
               'COKE': 'COKE_daily.csv',
               'PG': 'PG_daily.csv'}

# Training Inputs
trn_keys = ['DJI']#,'NDAQ','SP500']
training_range = ('2007-01-01','2008-12-31')
trn_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in training_range]

# Validation Inputs
val_keys = trn_keys
validation_range = ('2009-01-01', '2009-12-31')
val_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in validation_range]

# Testing Inputs
tst_keys = trn_keys#['AAPL','AMAZON','GOOGL','MSFT','FORD','JNJ','NEE','PFE','TSLA','COKE','PG']
testing_range = ('2010-01-01', '2010-12-31')
tst_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in testing_range]

window_size = 28 # Needs to match the size Timesnet is trained on
price_based_on = 'close'
columns = ['open','high','low','close','volume']

## RL Enviornment Generation

In [5]:
env_data = {}
env = {}

for stock, file in stock_inputs.items():
    if stock in set(trn_keys + val_keys + tst_keys):
        # Import
        df = cln.YAHOO_csv_input(file, input_url)
        data_dic = UniStockEnvDataStruct(df,price_based_on,window_size)
        env_data[stock] = data_dic
        env[stock] = ContinuousOHLCVEnv(name=stock,
                                        ohlcv_data = env_data[stock]['rw_raw_env'] ,
                                        stock_price_data= env_data[stock]['rw_raw_price_env'],
                                        commission_rate=0.005)

c:\Users\beckm\anaconda3\envs\MADDQN\Lib\site-packages\gymnasium\spaces\box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [6]:
trn_idx = {}
val_idx = {}
tst_idx = {}

for stock, file in stock_inputs.items():
    if stock in set(trn_keys + val_keys + tst_keys):
        if stock in trn_keys:
            trn_idx[stock] = env_data[stock].gen_rw_idxs(trn_dt_range)
        if stock in val_keys:
            val_idx[stock] = env_data[stock].gen_rw_idxs(val_dt_range)
        if stock in tst_keys:
            tst_idx[stock] = env_data[stock].gen_rw_idxs(tst_dt_range)


# Agent Setup

## Preprocessing

### TimesNet

In [7]:
train_tn_model = False
import_tn_model = False
import_tn_csvs = True
tn_path = pwd + '/gen_data/timesnet/'

#### Train

In [8]:
n_prediction = 5

if train_tn_model:
    
    model = TimesNet(h = n_prediction, # Forecast horizon
                    input_size = window_size, # Length of Batches
                    batch_size = 1, # Number of timeseries to predict
                    #futr_exog_list = remaining_columns,
                    hidden_size = 128, # Size of embedding for embedding and encoders,
                    dropout = 0.40, # Dropout for embeddings
                    conv_hidden_size = 3, # Channels for the inception block
                    top_k = 5, # Top num of periods from FFT considered
                    num_kernels = 13, # number of kernels for the inception block
                    encoder_layers = 3, # num of encoders
                    max_steps = 1000, # of training steps
                    early_stop_patience_steps = 10, #early stoppage on validation
                    val_check_steps = 100, # Val check every X steps,
                    windows_batch_size = 150, # Number of windows in training epoch,
                    num_workers_loader = 7,
                    learning_rate = 0.0003,
                    random_seed = RANDOM_SEED)

In [9]:
if train_tn_model:
  nf = NeuralForecast(models=[model], freq='d')
  results = {}
  for key in trn_keys:
    results[key] = nf.fit(df=env[key],val_size=0.2)

  nf.save(path= tn_path,
          model_index=None,
          overwrite=True,
          save_dataset=True)

#### Import Model

In [10]:
if import_tn_model:
# Define the correct path
  if IN_COLAB:
    
    model_path = os.path.join(os.getcwd(), 'gen_data', 'timesnet')

    # Ensure the directory and file exist
    if os.path.exists(model_path):
        nf = NeuralForecast.load(path=model_path)
    else:
        raise FileNotFoundError(f"Model path {model_path} does not exist.")

#### Import Timesnet CSVs

In [11]:
if import_tn_csvs:

    if IN_COLAB:
        # Input Data Location, File Name, Stock name for labels
        csv_path = 'https://raw.githubusercontent.com/CodeBeckZero/MADDQN/main/gen_data/csvs/'

    else:
        csv_path  = pwd +'/gen_data/csvs/'

    stock_tn ={'DJI':'DJI_tn.csv',
                'NDAQ': 'NDAQ_tn.csv',
                'SP500': 'SP500_tn.csv',
                'AAPL': 'AAPL_tn.csv',
                'AMZN': 'AMZN_tn.csv',
                'GOOGL': 'GOOGL_tn.csv',
                'MSFT': 'MSFT_tn.csv',
                'FORD': 'FORD_tn.csv',
                'JNJ': 'JNJ_tn.csv',
                'NEE': 'NEE_tn.csv',
                'PFE': 'PFE_tn.csv',
                'TSLA': 'TSLA_tn.csv',
                'COKE': 'COKE_tn.csv',
                'PG': 'PG_tn.csv'}

    timesnet = TimesNetProcessing(env_data)

## DDQN Setup

In [22]:
env_mod_func = {'train': timesnet.process,
               'import':timesnet.process,
               'csv':timesnet.csv_process,}

best_ddqn_agent = DdqnAgent(name='profit',
                        environment=None,
                        reward_function = future_profit,
                        reward_params = {'n':5},
                        env_state_mod_func = env_mod_func['csv'],     
                        input_size= 11,
                        hidden_size= 256, 
                        output_size=3, 
                        activation_function = nn.GELU(),
                        num_hidden_layers = 2,                  
                        buffer_size= 100, 
                        batch_size = 20,
                        opt_lr= 0.001,
                        alpha = 0.01,
                        gamma = 0.9,
                        opt_wgt_dcy = 0.0,
                        dropout_rate = 0.25,                
                        device = device)

## Train DDQN

In [13]:
def metric_function(env):
    metric = env.step_info[-1]['New Portfolio Value'] -  env.step_info[-1]['Portfolio Value']
    return metric

In [23]:
for key, rl_env in env.items():
    
    if key in trn_keys:
        timesnet.upload_csv(csv_path + stock_tn[key])
        env[key].add_agent(best_ddqn_agent.get_name())
        env[key].set_decision_agent(best_ddqn_agent.get_name())
        best_ddqn_agent.set_environment(rl_env)
        best_ddqn_agent.train(start_idx = trn_idx[key][0],
                    end_idx=trn_idx[key][1],
                    training_episodes= 100,
                    epsilon_decya_func= linear_decay,
                    initial_epsilon= 0.9,
                    final_epsilon= 0.1,
                    update_q_freq= 1,
                    update_tgt_freq= 10,
                    save_path = export_path,
                    val_start_idx = val_idx[key][0],
                    val_end_idx = val_idx[key][1],
                    metric_func= metric_function,
                    early_stop = True,
                    stop_metric = 'val_tot_r',
                    stop_patience = 7,
                    stop_delta = 0.001)
    
        ## Export Training Session Data to CSV
        ddqn_trn = best_ddqn_agent.get_training_episodic_data()
        ddqn_trn.to_csv('test.csv')
        display(ddqn_trn)
        env[key].remove_agent(best_ddqn_agent.get_name())

DJI ENV: Agent profit already exists
DJI ENV: Agent profit assigned as decision agent
profit: Training Initialized on DJI[0:476] -> Validation on DJI[504:728]
profit: EP 35 of 100 Finished -> ΔQ1 = 0.58, ΔQ2 = 0.47 | ∑R = 10.84, μR = 0.02 σR = 2.07 | Max: val_tot_r = 33611.96 -> EarlyStopping counter: 7 out of 7                                               .......
profit: Early Stoppage on EPIDSODE 35 -> Best QNet Loaded

profit: Training finished on DJI[0:476]


,trn_ep,tot_r,avg_r,std_r,Q1_loss,Q2_loss,epsilon,val_tot_r,val_avg_r,val_std_r,val_ror,val_comm_cost
0,1,-76.386731,-0.160476,1.911159,0.727446,0.262370,0.900,17414.92245,77.745190,795.589908,1.174149,6105.10755
1,2,-141.127786,-0.296487,2.412885,1.195485,1.874706,0.892,32694.13390,145.955955,1332.850322,1.332898,13334.56610
2,3,-70.314549,-0.147720,2.569280,18.036682,23.699015,0.884,46812.57355,208.984703,1381.601021,1.475076,5383.85645
3,4,-29.642242,-0.062274,2.359917,33.568947,14.859829,0.876,61030.53375,272.457740,1578.265792,1.617752,2531.51625
4,5,-45.801291,-0.096221,2.139400,16.756727,18.319683,0.868,30198.47760,134.814632,1400.320943,1.307942,1109.28240
5,6,-83.451113,-0.175317,2.487151,59.371460,58.004093,0.860,29987.37495,133.872210,1464.273564,1.305831,9525.33505
6,7,-153.867082,-0.323250,2.289338,11.931170,8.024136,0.852,-48213.10410,-215.237072,973.997686,0.517869,62867.86410
7,8,-116.206748,-0.244132,2.434116,4.453424,4.007975,0.843,-10014.65780,-44.708294,719.693677,0.899853,3459.89780
8,9,-73.354233,-0.154106,2.539044,3.362362,4.978491,0.835,-12650.29430,-56.474528,715.306641,0.873497,5138.03430
9,10,-162.378206,-0.341131,2.601068,2.198942,2.091537,0.827,-15045.84565,-67.168954,744.061891,0.849542,12504.37565


DJI ENV: Agent profit removed


In [24]:
display(best_ddqn_agent.get_step_data())
display(env['DJI'].get_step_data())

,profit Action,profit Action Type,profit Q_Val Sell,profit Q_Val Hold,profit Q_Val Buy,profit Reward
0,H,Best,0.686851,-0.012667,-3.216686,0.0000
1,H,Best,0.574000,-0.136944,-3.490445,0.0000
2,H,Best,0.646691,-0.084542,-3.853395,0.0000
3,H,Best,0.521288,0.117277,-0.883392,0.0000
4,H,Best,0.659478,0.072867,-2.322202,0.0000
...,...,...,...,...,...,...
219,H,Best,5.803301,7.839804,11.137906,18.1200
220,H,Best,2.866000,3.078868,4.513406,643.8000
221,H,Best,4.219491,5.383128,7.687745,323.8800
222,H,Best,5.176784,6.885295,9.818885,-20.0400


,Step,idx,Portfolio Value,Cash,Stock Value,Stock Holdings,Stock Price,State,Available Actions,Env Action,New Portfolio Value,New Cash,New Stock Value,New Stock Holdings,New Commission Cost,Total Commission Cost
0,1,504,100000.00000,100000.00000,0.00,0,7939.53,"([[8772.25, 9065.28, 8760.78, 9034.69, 2137000...","(H, B)",H,100000.00000,100000.00000,0.00,0,0.0000,0.00000
1,2,505,100000.00000,100000.00000,0.00,0,7932.76,"([[9027.13, 9034.37, 8892.36, 8952.89, 2337600...","(H, B)",H,100000.00000,100000.00000,0.00,0,0.0000,0.00000
2,3,506,100000.00000,100000.00000,0.00,0,7850.41,"([[8954.57, 9088.06, 8940.95, 9015.1, 21541000...","(H, B)",H,100000.00000,100000.00000,0.00,0,0.0000,0.00000
3,4,507,100000.00000,100000.00000,0.00,0,7552.60,"([[8996.94, 8996.94, 8719.92, 8769.7, 26671000...","(H, B)",H,100000.00000,100000.00000,0.00,0,0.0000,0.00000
4,5,508,100000.00000,100000.00000,0.00,0,7555.63,"([[8769.94, 8770.02, 8651.19, 8742.46, 2266200...","(H, B)",H,100000.00000,100000.00000,0.00,0,0.0000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,220,723,133241.91415,7662.75415,125597.28,12,10466.44,"([[10197.85, 10305.69, 10192.03, 10270.47, 167...","(S, H)",H,133260.03415,7662.75415,125597.28,12,0.0000,32977.89585
220,221,724,133260.03415,7662.75415,126241.08,12,10520.09,"([[10267.53, 10434.24, 10267.45, 10406.96, 202...","(S, H)",H,133903.83415,7662.75415,126241.08,12,0.0000,32977.89585
221,222,725,133903.83415,7662.75415,126564.96,12,10547.08,"([[10404.77, 10438.17, 10361.99, 10437.42, 158...","(S, H)",H,134227.71415,7662.75415,126564.96,12,0.0000,32977.89585
222,223,726,134227.71415,7662.75415,126544.92,12,10545.41,"([[10426.27, 10432.66, 10360.1, 10426.31, 1663...","(S, H)",H,134207.67415,7662.75415,126544.92,12,0.0000,32977.89585


## Hypertuning

In [ ]:
# For Objective function, need to create agent name before to link agent with enviornment
agent_name = 'REWARD_DDQN_AGENT'
agent_path = export_path + '/' + agent_name
metric = 'val_ror'

for key, env in environments.items():
  
        env.add_agent(agent_name)
        env.set_decision_agent(agent_name)

def objective(trial):
    
    # Define the hyperparameters to search over
    
    ## NN hyperparameters
    sug_hidden_layers = trial.suggest_int('hidden_layers', low=1, high=3)
    sug_hidden_size = trial.suggest_int('hidden_size', low=256, high=1280, step=64)
    sug_update_q_freq = trial.suggest_int('update_q_freq',low=1,high=5)
    sug_update_tgt_freq = trial.suggest_int('update_tgt_freq',low=10,high=50,step=10)
    
    ## Function Passing
    activation_functions = {
    'RELU': nn.ReLU(),
    'LRELU': nn.LeakyReLU(),
    'GELU': nn.GELU(),
    'TANH': nn.Tanh()
    }
    sug_activation_function_name = trial.suggest_categorical('activation_function', list(activation_functions.keys()))
    sug_activation_function = activation_functions[sug_activation_function_name]
    
    ## Optimizer hyperparameters
    sug_opt_lre = trial.suggest_float('opt_lre',0.0001,0.1,log=True)
    ## Memory Replay hyperparameters
    sug_buffer_size = trial.suggest_int('buffer_size',low=100,high=1500,step=100)
    sug_batch_size = trial.suggest_int('batch_size',low=10,high=150,step=10)

    # Saving Setup
    ## Current Parameter Values:
    cur_n_fcl = trial.params['hidden_layers']
    cur_fcl_size = trial.params['hidden_size']
    cur_q_freq = trial.params['update_q_freq']
    cur_tgt_freq = trial.params['update_tgt_freq']
    cur_act_func = trial.params['activation_function']
    cur_lre = decimal_to_text(trial.params['opt_lre'])
    cur_buf_size = trial.params['buffer_size']
    cur_bat_size = trial.params['batch_size']
    
    ## Create Notation for Hyperparameter Setup    
    test_name = (f'{cur_n_fcl}FC{cur_fcl_size}_{cur_act_func}_' +
                f'BT{cur_bat_size}BF{cur_buf_size}_Q{cur_q_freq}_' +
                f'TGT{cur_tgt_freq}_LR{cur_lre}')
    
    ## Create Dir to save results
    test_name_path =  agent_path + '/' + test_name 
    if not os.path.exists(test_name_path):
        os.makedirs(test_name_path)
        print(f"Directory '{test_name_path}' created successfully.")
    else:
        print(f"Directory '{test_name_path}' already exists.")    
    
    # Create Agent with hyperparameters  
    best_ddqn_agent = DdqnAgent(name=agent_name,
                        environment=None,
                        reward_function = future_profit,
                        reward_params = {'n':5},
                        env_state_mod_func = flatten_state,     
                        input_size= 13,
                        hidden_size= sug_hidden_size, 
                        output_size=3, 
                        activation_function = sug_activation_function,
                        num_hidden_layers = sug_hidden_layers,                  
                        buffer_size= sug_buffer_size, 
                        batch_size = sug_batch_size,
                        opt_lr= sug_opt_lre,
                        alpha = ALPHA,
                        gamma = GAMMA,
                        opt_wgt_dcy = 0.0,
                        dropout_rate = 0.25,                
                        device = device)

    # Training Model
    for key, env in environments.items():
        
        if key in trn_keys:
            
            best_ddqn_agent.set_environment(env)
            best_ddqn_agent.train(start_idx=training_range[0],
                        end_idx=training_range[1],
                        training_episodes= 100,
                        epsilon_decya_func= linear_decay,
                        initial_epsilon= 0.9,
                        final_epsilon= 0.1,
                        update_q_freq= sug_update_q_freq,
                        update_tgt_freq= sug_update_tgt_freq,
                        save_path = export_path,
                        val_start_idx = validation_range[0],
                        val_end_idx = validation_range[1],
                        early_stop = True,
                        stop_metric = metric,
                        stop_patience = 20,
                        stop_delta = 0.001)
        
            ## Export Training Session Data to CSV
            ddqn_trn = best_ddqn_agent.get_training_episodic_data()
            trn_df_file_name  = f'TRN-{key}' + test_name + '.csv'
            trn_df_save_path = test_name_path + '/' + trn_df_file_name
            ddqn_trn.to_csv(trn_df_save_path)
        
        
    
    # Test Model
    
    
    scores = []
    for key, env in environments.items():
    
        if key in tst_keys:
            
            best_ddqn_agent.set_environment(env)              
            best_ddqn_agent.test(start_idx = testing_range[0],
                        end_idx = testing_range[1], 
                        testing_episodes=1)

            ## Save Test Metric Result(s) into 
            ddqn_tst = best_ddqn_agent.get_testing_episodic_data()
            score = ddqn_tst['Total Reward'].mean()
            scores.append(score)
    
            ## Export Test data
            a = env.get_step_data()
            b = best_ddqn_agent.get_step_data()
            combined_df = pd.concat([a,b],axis=1)
            tst_df_file_name  = f'TST-{key}' + test_name + '.csv'
            trn_df_save_path = test_name_path + '/' + tst_df_file_name
            combined_df.to_csv(trn_df_save_path)

            ## Generate Trading Graphic
            tst_graph_file_name = trn_df_save_path[:-4] + '.png'
            agentperform.agent_stock_performance(env.stock_price_data[testing_range[0]:testing_range[1]],
                                                combined_df['Env Action'].to_numpy(),
                                                key,
                                                best_ddqn_agent.get_name(),
                                                display_graph=True,
                                                save_graphic=True,
                                                path_file=tst_graph_file_name)

    mean = np.mean(scores)
    return mean

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Best value: ", study.best_value)
print("Best params: ", study.best_params)